In [ ]:
import pandas as pd
# from mplcursors import cursor
import numpy as np
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import xarray as xr
import datetime
import os
import plotly.graph_objects as go
# import import_ipynb
from sklearn import linear_model
import statsmodels.api as sm
import glob
import warnings
import ee
import geopandas as gpd
ee.Initialize()
import geemap
import pickle
from scipy.optimize import curve_fit
from sklearn.metrics import mean_squared_error, r2_score
warnings.filterwarnings("ignore") 

In [ ]:
### If you dont want to execute the complete process and want the code of Robust AV Curve fitting take it from below commented 
### code. If you are stakeholde and want to implent it in your region then follow the notebook.

#############Code - Begins##################

# # Sort the DataFrame by 'Area' column
# Sorted_AV = AV_File.sort_values('Area')

# # # Calculate the cumulative maximum volume for each area
# Sorted_AV['CumulativeMaxVolume'] = Sorted_AV['Mean_Vol(km3)'].cummax()

# # Filter the DataFrame to include rows where Volume > CumulativeMaxVolume
# Sorted_AV = Sorted_AV[Sorted_AV['Mean_Vol(km3)'] >= Sorted_AV['CumulativeMaxVolume']]

# # Print the filtered DataFrame
# Sorted_AV = Sorted_AV.reset_index(drop=True)

#############Code - End ##################


In [ ]:
## Gauges data of each regime. Should contain gauge name and it lat and lon coordinates
NE_Quad_region = pd.read_csv(r'Path to Gauge lat lon csv files\Gauges_Intersection_NE_BD.csv')
NW_Quad_region = pd.read_csv(r'Path to Gauge lat lon csv files\Gauges_Intersection_NW_BD.csv')
SE_Quad_region = pd.read_csv(r'Path to Gauge lat lon csv files\Gauges_Intersection_SE_BD.csv')
SW_Quad_region = pd.read_csv(r'Path to Gauge lat lon csv files\Gauges_Intersection_SW_BD.csv')

In [ ]:
## Collects all the lat, lon and name of gauge in each regime together
regions = ['Gauges_Intersection_NE_BD.csv', 'Gauges_Intersection_NW_BD.csv', 'Gauges_Intersection_SE_BD.csv', 'Gauges_Intersection_SW_BD.csv']
names_NE = []
names_NW = []
names_SE = []
names_SW = []
names = [names_NE,names_NW,names_SE,names_SW]
for i in range(len(regions)):
    f = pd.read_csv(r'Path to Gauge lat lon csv files\\'+regions[i])
    for j in range(len(f)):
        names[i].append(f['Lake'][j].replace(' ','_'))  

In [ ]:
# Define a function to load and preprocess the data
def load_waterbody_data(file_path):
    print("Loading waterbody data:", file_path)
    data = pd.read_csv(file_path)
    data = data.dropna(subset = ['esm_mean']).reset_index(drop=True)
    m = np.quantile(data['esm_mean'],0.95)
    n = np.quantile(data['esm_mean'],0.05)
    print('Upper Quantile:', m)
    print('Lower Quantile:', n)
    data=data.loc[data['esm_mean']>=n]
    data=data.loc[data['esm_mean']<=m].reset_index(drop = True)
    
    m = np.quantile(data['esm_max'],0.95)
    n = np.quantile(data['esm_max'],0.05)
    print('Upper Quantile:', m)
    print('Lower Quantile:', n)
    data=data.loc[data['esm_max']>=n]
    data=data.loc[data['esm_max']<=m].reset_index(drop = True)
    
    m = np.quantile(data['esm_min'],0.95)
    n = np.quantile(data['esm_min'],0.05)
    print('Upper Quantile:', m)
    print('Lower Quantile:', n)
    data=data.loc[data['esm_min']>=n]
    data=data.loc[data['esm_min']<=m].reset_index(drop = True)
    
    data['Name'] = file_path.split('Ensemble_file_')[1].split(".csv")[0]
    print('Finished Preprocessing')
    return data


# waterbody_data = [load_waterbody_data(os.path.join(data_directory, file)) for file in csv_files]


In [ ]:
# Load the data from the CSV files
data_directory = r"Path to A-V data file obtained through existing gauges for all region or regimes"
waterbody_data_NE = []
waterbody_data_NW = []
waterbody_data_SE = []
waterbody_data_SW = []
waterbody_datas = [waterbody_data_NE,waterbody_data_NW,waterbody_data_SE,waterbody_data_SW]
files_list =[]
# csv_files = [f for f in os.listdir(data_directory) if f.endswith(".csv")]
for i in range(len(names)):
    for j in range(len(names[i])):
        print('Working on:', names[i][j])
        for file in glob.glob(r"Path to A-V data file obtained through existing gauges for all region or regimes\\"+'*'+names[i][j]+'*'+'.csv'):
            print(file)
            files_list.append(file)
            waterbody_datas[i].append(load_waterbody_data(file))
#         if os.path.exists(r"Path to A-V data file obtained through existing gauges for all region or regimes\\"+'*'+names[i][j]+'*'+'.csv') !=True:
#             print('Did not find:',names[i][j])

In [ ]:
## Appending all the pre-processed data
complete_waterdata_NE = []
complete_waterdata_NW = []
complete_waterdata_SE = []
complete_waterdata_SW = []
complete_waterdata = [complete_waterdata_NE,complete_waterdata_NW,complete_waterdata_SE,complete_waterdata_SW]
for i in range(len(complete_waterdata)):
    complete_waterdata[i].append(pd.concat([one_data for one_data in waterbody_datas[i]]))


In [ ]:
## Renaming the columns of each file and droping the duplicates
for i in range(len(complete_waterdata)):
    complete_waterdata[i][0]['Date'] = pd.to_datetime(complete_waterdata[i][0]['Date'])
    complete_waterdata[i][0] = complete_waterdata[i][0].sort_values(by = 'Area').reset_index(drop = True)
    complete_waterdata[i][0] = complete_waterdata[i][0].rename(columns = {'esm_mean':'Mean_Vol(km3)'})
    complete_waterdata[i][0] = complete_waterdata[i][0].rename(columns = {'esm_min':'Min_Vol(km3)'})
    complete_waterdata[i][0] = complete_waterdata[i][0].rename(columns = {'esm_max':'Max_Vol(km3)'})
    complete_waterdata[i][0]['Max_Vol(km3)'] = complete_waterdata[i][0]['Max_Vol(km3)']
    complete_waterdata[i][0] = complete_waterdata[i][0].drop_duplicates(subset = ['Date','Name'],keep = 'last').reset_index(drop = True)


In [ ]:
# Sort the DataFrame by 'Area' column
NE_Waterdata = complete_waterdata[0][0].sort_values('Area')

# # Calculate the cumulative maximum volume for each area
NE_Waterdata['CumulativeMaxVolume'] = NE_Waterdata['Mean_Vol(km3)'].cummax()

# Filter the DataFrame to include rows where Volume > CumulativeMaxVolume
NE_Waterdata = NE_Waterdata[NE_Waterdata['Mean_Vol(km3)'] >= NE_Waterdata['CumulativeMaxVolume']]



# Print the filtered DataFrame
NE_Waterdata = NE_Waterdata.reset_index(drop=True)

NE_Waterdata.head(2)


In [ ]:
fig,ax = plt.subplots(figsize = (10,6))
ax.plot(NE_Waterdata['Area'],NE_Waterdata['Mean_Vol(km3)']*1000,color = 'black',label = 'Robust Fitting')
ax.scatter(complete_waterdata[0][0]['Area'],complete_waterdata[0][0]['Mean_Vol(km3)']*1000,label = 'Citizen Science & Satellite Based Estimated Volume')
ax.set_xlabel('Area (km2)',fontweight = 'bold',fontsize = 14)
ax.set_ylabel('Volume (Million $m^3$)',fontweight = 'bold',fontsize = 14)
ax.set_title('A-V Curve for NE Bangladesh',fontweight = 'bold',fontsize = 17)
ax.legend()

### Doing the regression and implementing for other regions

In [ ]:
NE_Waterdata.head(2)

In [ ]:
#NE
x = NE_Waterdata[['Area',]]
y = NE_Waterdata['CumulativeMaxVolume']
 
# with statsmodels
# x = sm.add_constant(x) # adding a constant
 
model = sm.OLS(y, x).fit()
predictions = model.predict(x) 
 
print_model = model.summary()
print(print_model)

In [ ]:
#NW

# Sort the DataFrame by 'Area' column
NW_Waterdata = complete_waterdata[1][0].sort_values('Area')

# # Calculate the cumulative maximum volume for each area
NW_Waterdata['CumulativeMaxVolume'] = NW_Waterdata['Mean_Vol(km3)'].cummax()

# Filter the DataFrame to include rows where Volume > CumulativeMaxVolume
NW_Waterdata = NW_Waterdata[NW_Waterdata['Mean_Vol(km3)'] >= NW_Waterdata['CumulativeMaxVolume']]



# Print the filtered DataFrame
NW_Waterdata = NW_Waterdata.reset_index(drop=True)



x = NW_Waterdata[['Area',]]
y = NW_Waterdata['CumulativeMaxVolume']
 
# with statsmodels
# x = sm.add_constant(x) # adding a constant
 
model = sm.OLS(y, x).fit()
predictions = model.predict(x) 
 
print_model = model.summary()
print(print_model)
NW_Waterdata['pred_vol'] = predictions

from matplotlib.colors import ListedColormap
fig,ax = plt.subplots(figsize = (10,6))
# marker_shape = train_data['TPI'].astype('string')
# marker_shape = marker_shape.tolist() 
# colors = ['blue', 'green', 'red']
# cmap = ListedColormap(colors)
cmap='viridis'
ax.scatter(NW_Waterdata['Area'],NW_Waterdata['Mean_Vol(km3)']*1000,color = 'black',label = 'Robust Points')
ax.plot(NW_Waterdata['Area'],NW_Waterdata['pred_vol']*1000,color = 'blue',label = 'Regression Line')
ax.set_title('Robust A-V Curves', fontname="Times New Roman", fontweight="bold",fontsize = 18)
ax.set_ylabel('Volume of Water Stored ($Million$ $M^3$)',fontname="Times New Roman", fontweight="bold",fontsize = 13)
ax.set_xlabel('Area of Water Body ($Km^2$)',fontname="Times New Roman", fontweight="bold",fontsize = 13)
ax.legend()
# textstr = '\n'.join((
#     r'$R^2=%.2f$' % (model.rsquared, ),
#     r'Volume of water body = Area x 2.5262 + Precipitaion x 0.0024 + Temperature x (-0.1307) + TPI x 3.8387 + 1.3692'
#     ))
textstr = '\n'.join((
    r'$R^2=%.2f$' % (model.rsquared, ),
    r'$y =%.4f x$' % (model.params.Area)
    ))
props = dict(boxstyle='round', facecolor='grey', alpha=0.1)
ax.text(0.020, 0.85, textstr, transform=ax.transAxes, fontsize=14,
        verticalalignment='top', bbox=props)

In [ ]:
#SE

# Sort the DataFrame by 'Area' column
SE_Waterdata = complete_waterdata[2][0].sort_values('Area')

# # Calculate the cumulative maximum volume for each area
SE_Waterdata['CumulativeMaxVolume'] = SE_Waterdata['Mean_Vol(km3)'].cummax()

# Filter the DataFrame to include rows where Volume > CumulativeMaxVolume
SE_Waterdata = SE_Waterdata[SE_Waterdata['Mean_Vol(km3)'] >= SE_Waterdata['CumulativeMaxVolume']]



# Print the filtered DataFrame
SE_Waterdata = SE_Waterdata.reset_index(drop=True)



x = SE_Waterdata[['Area',]]
y = SE_Waterdata['CumulativeMaxVolume']
 
# with statsmodels
# x = sm.add_constant(x) # adding a constant
 
model = sm.OLS(y, x).fit()
predictions = model.predict(x) 
 
print_model = model.summary()
print(print_model)
SE_Waterdata['pred_vol'] = predictions

from matplotlib.colors import ListedColormap
fig,ax = plt.subplots(figsize = (10,6))
# marker_shape = train_data['TPI'].astype('string')
# marker_shape = marker_shape.tolist() 
# colors = ['blue', 'green', 'red']
# cmap = ListedColormap(colors)
cmap='viridis'
ax.scatter(SE_Waterdata['Area'],SE_Waterdata['Mean_Vol(km3)']*1000,color = 'black',label = 'Robust Points')
ax.plot(SE_Waterdata['Area'],SE_Waterdata['pred_vol']*1000,color = 'blue',label = 'Regression Line')
ax.set_title('Robust A-V Curves', fontname="Times New Roman", fontweight="bold",fontsize = 18)
ax.set_ylabel('Volume of Water Stored ($Million$ $M^3$)',fontname="Times New Roman", fontweight="bold",fontsize = 13)
ax.set_xlabel('Area of Water Body ($Km^2$)',fontname="Times New Roman", fontweight="bold",fontsize = 13)
ax.legend()
# textstr = '\n'.join((
#     r'$R^2=%.2f$' % (model.rsquared, ),
#     r'Volume of water body = Area x 2.5262 + Precipitaion x 0.0024 + Temperature x (-0.1307) + TPI x 3.8387 + 1.3692'
#     ))
textstr = '\n'.join((
    r'$R^2=%.2f$' % (model.rsquared, ),
    r'$y =%.4f x$' % (model.params.Area)
    ))
props = dict(boxstyle='round', facecolor='grey', alpha=0.1)
ax.text(0.020, 0.85, textstr, transform=ax.transAxes, fontsize=14,
        verticalalignment='top', bbox=props)

In [ ]:
#SW

# Sort the DataFrame by 'Area' column
SW_Waterdata = complete_waterdata[3][0].sort_values('Area')

# # Calculate the cumulative maximum volume for each area
SW_Waterdata['CumulativeMaxVolume'] = SW_Waterdata['Mean_Vol(km3)'].cummax()

# Filter the DataFrame to include rows where Volume > CumulativeMaxVolume
SW_Waterdata = SW_Waterdata[SW_Waterdata['Mean_Vol(km3)'] >= SW_Waterdata['CumulativeMaxVolume']]



# Print the filtered DataFrame
SW_Waterdata = SW_Waterdata.reset_index(drop=True)



x = SW_Waterdata[['Area',]]
y = SW_Waterdata['CumulativeMaxVolume']
 
# with statsmodels
# x = sm.add_constant(x) # adding a constant
 
model = sm.OLS(y, x).fit()
predictions = model.predict(x) 
 
print_model = model.summary()
print(print_model)
SW_Waterdata['pred_vol'] = predictions

from matplotlib.colors import ListedColormap
fig,ax = plt.subplots(figsize = (10,6))
# marker_shape = train_data['TPI'].astype('string')
# marker_shape = marker_shape.tolist() 
# colors = ['blue', 'green', 'red']
# cmap = ListedColormap(colors)
cmap='viridis'
ax.scatter(SW_Waterdata['Area'],SW_Waterdata['Mean_Vol(km3)']*1000,color = 'black',label = 'Robust Points')
ax.plot(SW_Waterdata['Area'],SW_Waterdata['pred_vol']*1000,color = 'blue',label = 'Regression Line')
ax.set_title('Robust A-V Curves', fontname="Times New Roman", fontweight="bold",fontsize = 18)
ax.set_ylabel('Volume of Water Stored ($Million$ $M^3$)',fontname="Times New Roman", fontweight="bold",fontsize = 13)
ax.set_xlabel('Area of Water Body ($Km^2$)',fontname="Times New Roman", fontweight="bold",fontsize = 13)
ax.legend()
# textstr = '\n'.join((
#     r'$R^2=%.2f$' % (model.rsquared, ),
#     r'Volume of water body = Area x 2.5262 + Precipitaion x 0.0024 + Temperature x (-0.1307) + TPI x 3.8387 + 1.3692'
#     ))
textstr = '\n'.join((
    r'$R^2=%.2f$' % (model.rsquared, ),
    r'$y =%.4f x$' % (model.params.Area)
    ))
props = dict(boxstyle='round', facecolor='grey', alpha=0.1)
ax.text(0.020, 0.85, textstr, transform=ax.transAxes, fontsize=14,
        verticalalignment='top', bbox=props)

In [ ]:
#NW

# Sort the DataFrame by 'Area' column
NE_Waterdata = complete_waterdata[0][0].sort_values('Area')

# # Calculate the cumulative maximum volume for each area
NE_Waterdata['CumulativeMaxVolume'] = NE_Waterdata['Mean_Vol(km3)'].cummax()

# Filter the DataFrame to include rows where Volume > CumulativeMaxVolume
NE_Waterdata = NE_Waterdata[NE_Waterdata['Mean_Vol(km3)'] >= NE_Waterdata['CumulativeMaxVolume']]



# Print the filtered DataFrame
NE_Waterdata = NE_Waterdata.reset_index(drop=True)



x = NE_Waterdata[['Area',]]
y = NE_Waterdata['CumulativeMaxVolume']
 
# with statsmodels
# x = sm.add_constant(x) # adding a constant
 
model = sm.OLS(y, x).fit()
predictions = model.predict(x) 
 
print_model = model.summary()
print(print_model)
NE_Waterdata['pred_vol'] = predictions

from matplotlib.colors import ListedColormap
fig,ax = plt.subplots(figsize = (10,6))
# marker_shape = train_data['TPI'].astype('string')
# marker_shape = marker_shape.tolist() 
# colors = ['blue', 'green', 'red']
# cmap = ListedColormap(colors)
cmap='viridis'
ax.scatter(NE_Waterdata['Area'],NE_Waterdata['Mean_Vol(km3)']*1000,color = 'black',label = 'Robust Points')
ax.plot(NE_Waterdata['Area'],NE_Waterdata['pred_vol']*1000,color = 'blue',label = 'Regression Line')
ax.set_title('Robust A-V Curves', fontname="Times New Roman", fontweight="bold",fontsize = 18)
# textstr = '\n'.join((
#     r'$R^2=%.2f$' % (model.rsquared, ),
#     r'Volume of water body = Area x 2.5262 + Precipitaion x 0.0024 + Temperature x (-0.1307) + TPI x 3.8387 + 1.3692'
#     ))
textstr = '\n'.join((
    r'$R^2=%.2f$' % (model.rsquared, ),
    r'$y =%.4f x$' % (model.params.Area)
    ))
props = dict(boxstyle='round', facecolor='grey', alpha=0.1)
ax.text(0.020, 0.85, textstr, transform=ax.transAxes, fontsize=14,
        verticalalignment='top', bbox=props)